In [4]:
using JuMP, HiGHS

# Define parameters
l = 2  # Number of lanes
v = 5  # Total number of bus slots
bus_types = ["A", "B", "C"]  # Types of buses
#b = Dict("A" => 5, "B" => 4, "C" => 3)  # Total buses of each type
#b = Dict("A" => 3, "B" => 2, "C" => 1)  # Total buses of each type
b = Dict("A" => 4, "B" => 3, "C" => 3)  # Total buses of each type
n = sum(values(b))
max_deviation = 1
n

10

In [5]:
#Works also for calculating number of buses departed
function calculate_no_of_buses_arrived(arrivals, bus_type)
    buses_arrived = zeros(Int, length(arrivals))
    indices_with_arrival = Int[]
    for i in 1:length(arrivals)
        buses_arrived[i] = (i > 1 ? buses_arrived[i-1] : 0) + (arrivals[i] == bus_type ? 1 : 0)
        if arrivals[i] == bus_type
            push!(indices_with_arrival, i)  # Add the index to the list of indices with arrival
        end
        #println("At index $i: Number of $bus_type buses arrived = $buses_arrived[i]")
    end
    buses_arrived_at_indices = buses_arrived
    #buses_arrived_at_indices = [buses_arrived[i] for i in indices_with_arrival]
    return indices_with_arrival, buses_arrived_at_indices
end

function compute_max_min_arrivals(arrivals, bus_type, max_deviation, max_or_min)
    
    if max_or_min == "max"
        # Move the indices to the left by `max_deviation` steps, one step at a time
        for i in 1:length(arrivals)
            if arrivals[i] == bus_type
                # Move step by step by max_deviation steps to the left
                for j in 1:max_deviation
                    if i > 1
                        # Swap the element with the one on the left
                        arrivals[i], arrivals[i-1] = arrivals[i-1], arrivals[i]
                        #println("Moved element at index $i to index $(i-1): ", arrivals)
                        i -= 1  # Move to the next index left
                    end
                end
            end
        end
        indices_with_arrival, buses_arrived_at_indices = calculate_no_of_buses_arrived(arrivals, bus_type)
    end
    
    if max_or_min == "min"
        # Move the indices to the right by `max_deviation` steps, one step at a time
        for i in length(arrivals):-1:1
            if arrivals[i] == bus_type
                # Move step by step by max_deviation steps to the right
                for j in 1:max_deviation
                    if i < length(arrivals)
                        # Swap the element with the one on the right
                        arrivals[i], arrivals[i+1] = arrivals[i+1], arrivals[i]
                        #println("Moved element at index $i to index $(i+1): ", arrivals)
                        i += 1  # Move to the next index right
                    end
                end
            end
        end
        indices_with_arrival, buses_arrived_at_indices = calculate_no_of_buses_arrived(arrivals, bus_type)
    end

    return indices_with_arrival, buses_arrived_at_indices
end


function compute_departure_indices(departures, bus_type)
    indices_with_departure, buses_departed_at_indices = calculate_no_of_buses_arrived(departures, bus_type)
    return indices_with_departure, buses_departed_at_indices
end

# Example usage:

# Define your planned arrival scenario (single scenario now)
#arrivals = ["A", "B", "B", "C", "A", "B", "A", "C", "A", "B", "C", "A"]
#departures = ["A", "B", "B", "C", "A", "B", "A", "C", "A", "B", "C", "A"]
#arrivals = ["B", "A", "A", "C", "B", "A"]
#departures = ["B", "A", "A", "C", "B", "A"]
arrivals = ["A", "A", "A", "B", "B", "A", "B", "C", "C", "C"]
departures = ["A", "A", "A", "B", "B", "A", "B", "C", "C", "C"]
println(arrivals)
println(departures)
println("")

#compute_max_min_arrivals(arrivals, "B", 1, "max")

indices_high = Dict()
a_high = Dict()

indices_low = Dict()
a_low = Dict()

departure_indices = Dict()
no_of_departed = Dict()

for t in bus_types
    arrivals_copy = copy(arrivals)
    indices_low[t], a_low[t] = compute_max_min_arrivals(arrivals_copy, t, max_deviation, "min")
end

for t in bus_types
    arrivals_copy = copy(arrivals)
    indices_high[t], a_high[t] = compute_max_min_arrivals(arrivals_copy, t, max_deviation, "max")
end

for t in bus_types
    departures_copy = copy(departures)
    departure_indices[t], no_of_departed[t] = compute_departure_indices(departures_copy, t)
end

# Print the results
println("Indices: ", indices_high)
println("a_high: ", a_high)
println("")
println("Indices: ", indices_low)
println("a_low: ", a_low)
println("")
println("Indices: ", departure_indices)
println("no_of_departed: ", no_of_departed)


["A", "A", "A", "B", "B", "A", "B", "C", "C", "C"]
["A", "A", "A", "B", "B", "A", "B", "C", "C", "C"]

Indices: Dict{Any, Any}("B" => [3, 4, 6], "A" => [1, 2, 3, 5], "C" => [7, 8, 9])
a_high: Dict{Any, Any}("B" => [0, 0, 1, 2, 2, 3, 3, 3, 3, 3], "A" => [1, 2, 3, 3, 4, 4, 4, 4, 4, 4], "C" => [0, 0, 0, 0, 0, 0, 1, 2, 3, 3])

Indices: Dict{Any, Any}("B" => [5, 6, 8], "A" => [2, 3, 4, 7], "C" => [8, 9, 10])
a_low: Dict{Any, Any}("B" => [0, 0, 0, 0, 1, 2, 2, 3, 3, 3], "A" => [0, 1, 2, 3, 3, 3, 4, 4, 4, 4], "C" => [0, 0, 0, 0, 0, 0, 0, 1, 2, 3])

Indices: Dict{Any, Any}("B" => [4, 5, 7], "A" => [1, 2, 3, 6], "C" => [8, 9, 10])
no_of_departed: Dict{Any, Any}("B" => [0, 0, 0, 1, 2, 2, 3, 3, 3, 3], "A" => [1, 2, 3, 3, 3, 4, 4, 4, 4, 4], "C" => [0, 0, 0, 0, 0, 0, 0, 1, 2, 3])


In [6]:
# Function to generate all admissible patterns with an indicator
function generate_patterns(v, bus_types)
    patterns = []
    pattern_types = Dict()  # Store whether pattern is one-block (1) or two-block (2)
    exit_block = Dict()
    entry_block = Dict()

    # One-block patterns (cover all bus types in dict)
    for t in bus_types
        pattern = [(t, v)]
        push!(patterns, pattern)
        pattern_types[length(patterns)] = 1  # Mark as one-block

        # Exit dict: Only store nonzero values
        exit_dict = Dict(bt => v for bt in bus_types if bt == t)
        entry_dict = Dict()  # Empty because no entry occurs in a one-block pattern

        exit_block[length(patterns)] = exit_dict
        entry_block[length(patterns)] = entry_dict
    end

    # Two-block patterns
    for (i, t1) in enumerate(bus_types), (j, t2) in enumerate(bus_types)
        if i != j  # Ensure different types
            for s1 in 1:(v-1)  # Exit block size
                s2 = v - s1  # Entry block size
                pattern = [(t1, s1), (t2, s2)]
                push!(patterns, pattern)
                pattern_types[length(patterns)] = 2  # Mark as two-block

                # Only store nonzero values in dicts
                exit_dict = Dict(t1 => s1)
                entry_dict = Dict(t2 => s2)

                exit_block[length(patterns)] = exit_dict
                entry_block[length(patterns)] = entry_dict
            end
        end
    end

    return patterns, pattern_types, exit_block, entry_block
end

# Generate patterns and indicators
P, pattern_types, exit_block, entry_block = generate_patterns(v, bus_types)

# Print patterns with their types
for i in 1:length(P)
    println("Pattern $i: ", P[i], " - Type: ", pattern_types[i], " - Exit block: ", exit_block[i], " - Entry block: ", entry_block[i])
end

Pattern 1: [("A", 5)] - Type: 1 - Exit block: Dict("A" => 5) - Entry block: Dict{Any, Any}()
Pattern 2: [("B", 5)] - Type: 1 - Exit block: Dict("B" => 5) - Entry block: Dict{Any, Any}()
Pattern 3: [("C", 5)] - Type: 1 - Exit block: Dict("C" => 5) - Entry block: Dict{Any, Any}()
Pattern 4: [("A", 1), ("B", 4)] - Type: 2 - Exit block: Dict("A" => 1) - Entry block: Dict("B" => 4)
Pattern 5: [("A", 2), ("B", 3)] - Type: 2 - Exit block: Dict("A" => 2) - Entry block: Dict("B" => 3)
Pattern 6: [("A", 3), ("B", 2)] - Type: 2 - Exit block: Dict("A" => 3) - Entry block: Dict("B" => 2)
Pattern 7: [("A", 4), ("B", 1)] - Type: 2 - Exit block: Dict("A" => 4) - Entry block: Dict("B" => 1)
Pattern 8: [("A", 1), ("C", 4)] - Type: 2 - Exit block: Dict("A" => 1) - Entry block: Dict("C" => 4)
Pattern 9: [("A", 2), ("C", 3)] - Type: 2 - Exit block: Dict("A" => 2) - Entry block: Dict("C" => 3)
Pattern 10: [("A", 3), ("C", 2)] - Type: 2 - Exit block: Dict("A" => 3) - Entry block: Dict("C" => 2)
Pattern 11: [

In [7]:
# Initialize JuMP model
model = Model(HiGHS.Optimizer)

# Define decision variables for each pattern
@variable(model, X[1:length(P)] >= 0, Int)  # One variable per pattern
@variable(model, Y[1:length(P), 1:n] >= 0, Int)  # Integer variable for assignments
@variable(model, Z[1:length(P), 1:n] >= 0, Int)  # Integer variable for assignments

27×10 Matrix{VariableRef}:
 Z[1,1]   Z[1,2]   Z[1,3]   Z[1,4]   …  Z[1,7]   Z[1,8]   Z[1,9]   Z[1,10]
 Z[2,1]   Z[2,2]   Z[2,3]   Z[2,4]      Z[2,7]   Z[2,8]   Z[2,9]   Z[2,10]
 Z[3,1]   Z[3,2]   Z[3,3]   Z[3,4]      Z[3,7]   Z[3,8]   Z[3,9]   Z[3,10]
 Z[4,1]   Z[4,2]   Z[4,3]   Z[4,4]      Z[4,7]   Z[4,8]   Z[4,9]   Z[4,10]
 Z[5,1]   Z[5,2]   Z[5,3]   Z[5,4]      Z[5,7]   Z[5,8]   Z[5,9]   Z[5,10]
 Z[6,1]   Z[6,2]   Z[6,3]   Z[6,4]   …  Z[6,7]   Z[6,8]   Z[6,9]   Z[6,10]
 Z[7,1]   Z[7,2]   Z[7,3]   Z[7,4]      Z[7,7]   Z[7,8]   Z[7,9]   Z[7,10]
 Z[8,1]   Z[8,2]   Z[8,3]   Z[8,4]      Z[8,7]   Z[8,8]   Z[8,9]   Z[8,10]
 Z[9,1]   Z[9,2]   Z[9,3]   Z[9,4]      Z[9,7]   Z[9,8]   Z[9,9]   Z[9,10]
 Z[10,1]  Z[10,2]  Z[10,3]  Z[10,4]     Z[10,7]  Z[10,8]  Z[10,9]  Z[10,10]
 Z[11,1]  Z[11,2]  Z[11,3]  Z[11,4]  …  Z[11,7]  Z[11,8]  Z[11,9]  Z[11,10]
 Z[12,1]  Z[12,2]  Z[12,3]  Z[12,4]     Z[12,7]  Z[12,8]  Z[12,9]  Z[12,10]
 Z[13,1]  Z[13,2]  Z[13,3]  Z[13,4]     Z[13,7]  Z[13,8]  Z[13,9]  Z[1

In [8]:
# Objective: Minimize two-block patterns
@objective(model, Min, sum(X[i] for i in 1:length(P) if pattern_types[i] == 2))

X[4] + X[5] + X[6] + X[7] + X[8] + X[9] + X[10] + X[11] + X[12] + X[13] + X[14] + X[15] + X[16] + X[17] + X[18] + X[19] + X[20] + X[21] + X[22] + X[23] + X[24] + X[25] + X[26] + X[27]

In [9]:
# Constraint: Total lanes must match v
@constraint(model, total_lanes, sum(X[i] for i in 1:length(P)) == l)

total_lanes : X[1] + X[2] + X[3] + X[4] + X[5] + X[6] + X[7] + X[8] + X[9] + X[10] + X[11] + X[12] + X[13] + X[14] + X[15] + X[16] + X[17] + X[18] + X[19] + X[20] + X[21] + X[22] + X[23] + X[24] + X[25] + X[26] + X[27] = 2

In [10]:
# Constraints: Satisfy total bus requirements per type
for t in bus_types
    @constraint(model, sum((get(exit_block[i], t, 0) + get(entry_block[i], t, 0)) * X[i] for i in 1:length(P)) == b[t])
end

In [11]:
# Constraint (6)
for t in bus_types
    for i in indices_high[t]  # Loop through indices for each bus type
        @constraint(model, 
            sum(get(exit_block[p], t, 0) * X[p] for p in 1:length(P)) +
            sum(get(entry_block[p], t, 0) * Y[p, i] for p in 1:length(P) if pattern_types[p] == 2)
            >= a_high[t][i]  # Access a_high dictionary correctly for the specific bus type and index
        )
    end
end

In [12]:
# Constraint (7)
for t in bus_types
    for i in union([indices_low[t′] for t′ in bus_types if t′ ≠ t]...)  # Collect indices from all other bus types
        @constraint(model, 
            sum(get(exit_block[p], t, 0) * Y[p, i] for p in 1:length(P) if pattern_types[p] == 2)
            <= a_low[t][i]  # Access a_low dictionary correctly for the specific bus type and index
        )
    end
end

In [13]:
# Constraint (8)
for (idx, p) in enumerate(P)  # idx is the position in P, p is the value in P
    if pattern_types[idx] == 2
        for i in 1:n-1
            @constraint(model, Y[idx, i] <= Y[idx, i+1])
        end
    end
end

In [14]:
# Constraint (9)
for (idx, p) in enumerate(P)  # idx is the position in P, p is the value in P
    if pattern_types[idx] == 2
        @constraint(model, Y[idx, n] <= X[idx])
    end
end

In [15]:
# Constraint (10)
for t in bus_types
    for i in departure_indices[t]  # Loop through indices for each bus type
        @constraint(model, 
            sum(get(exit_block[p], t, 0) * X[p] for p in 1:length(P)) +
            sum(get(entry_block[p], t, 0) * Z[p, i] for p in 1:length(P) if pattern_types[p] == 2)
            >= no_of_departed[t][i]  # Access a_high dictionary correctly for the specific bus type and index
        )
    end
end

In [16]:
# Constraint (11)
for t in bus_types
    for i in union([departure_indices[t′] for t′ in bus_types if t′ ≠ t]...)  # Collect indices from all other bus types
        @constraint(model, 
            sum(get(exit_block[p], t, 0) * Z[p, i] for p in 1:length(P) if pattern_types[p] == 2)
            <= no_of_departed[t][i]  # Access a_low dictionary correctly for the specific bus type and index
        )
    end
end

In [17]:
# Constraint (12)
for (idx, p) in enumerate(P)  # idx is the position in P, p is the value in P
    if pattern_types[idx] == 2
        for j in 1:n-1
            @constraint(model, Z[idx, j] <= Z[idx, j+1])
        end
    end
end

In [18]:
# Constraint (13)
for (idx, p) in enumerate(P)  # idx is the position in P, p is the value in P
    if pattern_types[idx] == 2
        @constraint(model, Z[idx, n] <= X[idx])
    end
end

In [19]:
optimize!(model)

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 4e+00]
Presolving model
205 rows, 157 cols, 596 nonzeros  0s
101 rows, 74 cols, 391 nonzeros  0s
87 rows, 60 cols, 363 nonzeros  0s
77 rows, 5 cols, 79 nonzeros  0s
5 rows, 3 cols, 11 nonzeros  0s
3 rows, 3 cols, 7 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Tim

In [483]:
JuMP.value.(X)

27-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [484]:
P

27-element Vector{Any}:
 [("A", 5)]
 [("B", 5)]
 [("C", 5)]
 [("A", 1), ("B", 4)]
 [("A", 2), ("B", 3)]
 [("A", 3), ("B", 2)]
 [("A", 4), ("B", 1)]
 [("A", 1), ("C", 4)]
 [("A", 2), ("C", 3)]
 [("A", 3), ("C", 2)]
 [("A", 4), ("C", 1)]
 [("B", 1), ("A", 4)]
 [("B", 2), ("A", 3)]
 ⋮
 [("B", 1), ("C", 4)]
 [("B", 2), ("C", 3)]
 [("B", 3), ("C", 2)]
 [("B", 4), ("C", 1)]
 [("C", 1), ("A", 4)]
 [("C", 2), ("A", 3)]
 [("C", 3), ("A", 2)]
 [("C", 4), ("A", 1)]
 [("C", 1), ("B", 4)]
 [("C", 2), ("B", 3)]
 [("C", 3), ("B", 2)]
 [("C", 4), ("B", 1)]

In [22]:
JuMP.value.(Y)

15×6 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [23]:
JuMP.value.(Z)

15×6 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [142]:
print(model)

In [120]:
Base.displaysize(stdout) = (1000, 1000)

In [121]:
open("full_model_output.txt", "w") do io
    print(io, model)  # Prints everything into a text file
end

In [118]:
show(model, context=:limit => false)

LoadError: MethodError: no method matching show(::Model; context::Pair{Symbol, Bool})
This method may not support any kwargs.

[0mClosest candidates are:
[0m  show(::Any)[91m got unsupported keyword argument "context"[39m
[0m[90m   @[39m [90mBase[39m [90m[4mshow.jl:479[24m[39m
[0m  show([91m::IOContext[39m, [91m::JuMP.Containers.SparseAxisArray[39m)[91m got unsupported keyword argument "context"[39m
[0m[90m   @[39m [35mJuMP[39m [90m/opt/julia/packages/JuMP/CU7H5/src/Containers/[39m[90m[4mSparseAxisArray.jl:456[24m[39m
[0m  show([91m::IO[39m, [91m::Core.TypeofBottom[39m)[91m got unsupported keyword argument "context"[39m
[0m[90m   @[39m [90mBase[39m [90m[4mshow.jl:566[24m[39m
[0m  ...


In [31]:
for i in 1:n-1
    print(i)
end

12345

In [22]:
P

15-element Vector{Any}:
 [("A", 3)]
 [("B", 3)]
 [("C", 3)]
 [("A", 1), ("B", 2)]
 [("A", 2), ("B", 1)]
 [("A", 1), ("C", 2)]
 [("A", 2), ("C", 1)]
 [("B", 1), ("A", 2)]
 [("B", 2), ("A", 1)]
 [("B", 1), ("C", 2)]
 [("B", 2), ("C", 1)]
 [("C", 1), ("A", 2)]
 [("C", 2), ("A", 1)]
 [("C", 1), ("B", 2)]
 [("C", 2), ("B", 1)]

In [14]:
# Constraint (7) - Modified
for t in bus_types
    println("")
    println(t)
    for i in union([indices_low[t′] for t′ in bus_types if t′ ≠ t]...)  # Collect indices from all other bus types
        println(i)
    end
end


A
1
2
5
3

B
1
4
3

C
1
4
2
5


In [15]:
exit_block[3]["A"]

1

In [57]:
# Constraint (6)
for t in bus_types, i in 1:l
    @constraint(model, sum(bus_distribution[p][t] * X[p] for p in 1:length(P) if t in bus_distribution[p]) >= a[t, i]) # (6)
end

LoadError: AbstractDict collections only contain Pairs;
Either look for e.g. A=>B instead, or use the `keys` or `values`
function if you are looking for a key or value respectively.

In [ ]:
for t in bus_types, i in 1:l
    @constraint(model, sum(bus_distribution[p][t] * Y[p, i] for p in 1:length(P) if t in bus_distribution[p]) <= a[t, i]) # (7)
end

for p in 1:length(P), i in 1:(l-1)
    @constraint(model, Y[p, i] <= Y[p, i+1]) # (8)
end

for p in 1:length(P)
    @constraint(model, Y[p, l] <= X[p]) # (9)
end

In [ ]:
# Solve the model
optimize!(model)

# Print results
println("Optimal assignment of patterns:")
for i in 1:length(P)
    println("Pattern ", i, ": X = ", value(X[i]))
end

In [34]:
@constraint(model, total_lanes, sum(X[i] for i in 1:length(P)) == v)

total_lanes : X[1] + X[2] + X[3] + X[4] + X[5] + X[6] + X[7] + X[8] + X[9] + X[10] = 3

In [35]:
for t in bus_types
    @constraint(model, sum(get(bus_distribution[i], t, 0) * X[i] for i in 1:length(P)) == b[t])
end

In [36]:
get(bus_distribution[3], "A", 0)

1

In [25]:
bus_types

2-element Vector{String}:
 "A"
 "B"